In [1]:
import os
import pickle
import cv2
import numpy as np
from insightface.app import FaceAnalysis

# Path to the main faces folder
faces_dir = "faces/"

# Initialize InsightFace with buffalo_l
face_app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
face_app.prepare(ctx_id=0, det_size=(640, 640))

# Dictionary to hold embeddings
known_faces = {}

# Ensure the faces directory exists
if not os.path.exists(faces_dir):
    print(f"Error: Directory {faces_dir} does not exist.")
    exit()

print(f"Found folders: {os.listdir(faces_dir)}")

# Loop through each person folder
for person_name in os.listdir(faces_dir):
    person_path = os.path.join(faces_dir, person_name)

    if not os.path.isdir(person_path):
        print(f"Skipping {person_name} (not a directory)")
        continue

    print(f"Processing {person_name}...")

    embeddings = []
    img_files = [f for f in os.listdir(person_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:6]  # Max 6 images

    if not img_files:
        print(f"  No valid images found for {person_name}")
        continue

    for img_file in img_files:
        img_path = os.path.join(person_path, img_file)
        try:
            print(f"  Encoding {img_file}...")
            # Load and preprocess image
            img = cv2.imread(img_path)
            if img is None:
                print(f"    Failed to load {img_file}: Unable to read image")
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            # Generate embedding
            faces = face_app.get(img_rgb)
            if faces:
                embedding = faces[0].embedding  # buffalo_l embeddings are normalized
                if embedding.shape == (512,):  # buffalo_l embedding size
                    embeddings.append(embedding)
                else:
                    print(f"    Invalid embedding shape for {img_file}: {embedding.shape}")
            else:
                print(f"    No faces detected in {img_file}")
        except Exception as e:
            print(f"    Failed to process {img_file}: {e}")

    if embeddings:
        print(f"  {len(embeddings)} images successfully processed for {person_name}")
        # Average embeddings
        avg_embedding = np.mean(embeddings, axis=0)
        # Normalize the embedding
        norm = np.linalg.norm(avg_embedding)
        if norm > 0:
            avg_embedding = avg_embedding / norm
        known_faces[person_name] = avg_embedding
    else:
        print(f"  No valid embeddings for {person_name}")

# Save to disk
if known_faces:
    with open("known_faces.pkl", "wb") as f:
        pickle.dump(known_faces, f)
    print("Embeddings saved to 'known_faces.pkl'")
else:
    print("No embeddings generated. Check images and paths.")

download_path: C:\Users\SanJos/.insightface\models\buffalo_l


100%|██████████| 281857/281857 [00:11<00:00, 25538.25KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112]

In [ ]:
import pickle
import cv2
import numpy as np
from ultralytics import YOLO
from insightface.app import FaceAnalysis
from scipy.spatial.distance import cosine
import time
from datetime import datetime
import pandas as pd

# Load known face embeddings
with open("known_faces.pkl", "rb") as f:
    known_faces = pickle.load(f)

# Initialize YOLOv8 for face detection
yolo_model = YOLO("yolov11n-face.pt")

# Initialize InsightFace for face recognition
face_app = FaceAnalysis(name="buffalo_l", providers=["CUDAExecutionProvider"])
face_app.prepare(ctx_id=0, det_size=(640, 640))

# Parameters
threshold = 0.5  # Tuned for buffalo_l cosine distance
SKIP_FRAMES = 10  # Faster than DeepFace
CACHE_RESET_INTERVAL = 30  # Reset cache every 30 frames
frame_count = 0
cache = {}
timers = {}
last_seen = {}
session_active = {}

# Start time
start_time = datetime.now()
print(f"Session started at {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# Load video file
video_path = r"C:\Users\SanJos\OneDrive\Desktop\face_recognition_with_active_window_time\vid02.mp4"
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    display_frame = frame.copy()
    current_time = time.time()

    # Reset cache periodically
    if frame_count % CACHE_RESET_INTERVAL == 0:
        cache.clear()
        print(f"Cache cleared at frame {frame_count}")

    # Face detection with YOLOv11
    results = yolo_model(frame, imgsz=640, conf=0.5)
    faces = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes
    seen_this_frame = set()

    for i, (x1, y1, x2, y2) in enumerate(faces):
        face_id = f"face_{i}"
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # Add padding to bounding box (30% of face size)
        padding = int(max(x2 - x1, y2 - y1) * 0.3)
        x1 = max(x1 - padding, 0)
        y1 = max(y1 - padding, 0)
        x2 = min(x2 + padding, frame.shape[1])
        y2 = min(y2 + padding, frame.shape[0])

        face_img = frame[y1:y2, x1:x2]
        print(f"Frame {frame_count}, Face {face_id} size: {face_img.shape}")

        if frame_count % SKIP_FRAMES == 0 or face_id not in cache:
            try:
                if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                    face_img = cv2.resize(face_img, (112, 112))  # Resize for InsightFace
                    face_img_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
                    faces_detected = face_app.get(face_img_rgb)
                    print(f"Faces detected by InsightFace: {len(faces_detected)}")
                    if faces_detected:
                        embedding = faces_detected[0].embedding  # Normalized
                        name = "Unknown"
                        min_dist = 1.0  # Track lowest distance

                        for person, ref_embedding in known_faces.items():
                            dist = cosine(embedding, ref_embedding)  # Cosine distance
                            print(f"Distance to {person}: {dist}")
                            if dist < threshold and dist < min_dist:
                                name = person
                                min_dist = dist
                        cache[face_id] = name
                    else:
                        cache[face_id] = "Unknown"
                else:
                    cache[face_id] = "Unknown"
            except Exception as e:
                print(f"Error processing face {face_id}: {e}")
                cache[face_id] = "Unknown"

        name = cache.get(face_id, "Detecting...")
        seen_this_frame.add(name)

        # Timer update logic
        if name != "Unknown":
            if not session_active.get(name, False):
                session_active[name] = True
                last_seen[name] = current_time
            else:
                duration = current_time - last_seen[name]
                timers[name] = timers.get(name, 0) + duration
                last_seen[name] = current_time

        # Draw bounding box and label
        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)

        # Draw timer (position below if near top)
        timer_display = f"{name}"
        if name in timers:
            timer_display += f" ({int(timers[name])}s)"
        text_y = y1 - 10 if y1 > 30 else y2 + 30  # Move below if near top
        cv2.putText(display_frame, timer_display, (x1, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    # Pause timers for absent users
    for name in session_active:
        if session_active[name] and name not in seen_this_frame:
            session_active[name] = False

    cv2.imshow("Face Recognition with Timer", display_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\SanJos/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112]